## Deliverable 2. Create a Customer Travel Destinations Map.

In [41]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import csv
import numpy as np

# Import API key
import sys
sys.path.insert(1,'..')
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [42]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Current Description,Date
0,0,Saskylakh,71.9167,114.0833,39.24,56,77,4.59,RU,broken clouds,2021-06-06 22:35:33
1,1,Mataura,-46.1927,168.8643,49.48,77,59,3.38,NZ,broken clouds,2021-06-06 22:34:15
2,2,Port Macquarie,-31.4333,152.9167,50.90,60,5,4.45,AU,clear sky,2021-06-06 22:31:26
3,3,Dera Bugti,29.0307,69.1510,87.87,47,0,5.08,PK,clear sky,2021-06-06 22:35:34
4,4,Port Hardy,50.6996,-127.4199,55.58,71,75,14.97,CA,broken clouds,2021-06-06 22:34:21


In [43]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
print("What is your preference for the minimum temperature(°F)?")
min_temp=float(input("min_temp: "))

print("What is your preference for the maximum temperature(°F)?")
max_temp=float(input("max_temp: "))


print(f"User's Preferred Temperatures: {min_temp}°F to {max_temp}°F")

What is your preference for the minimum temperature(°F)?
What is your preference for the maximum temperature(°F)?
User's Preferred Temperatures: 60.0°F to 70.0°F


In [44]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.

pref_cities = city_data_df.loc[(city_data_df["Max Temp"] >= min_temp) & (city_data_df["Max Temp"]<=max_temp)]

In [45]:
# 4a. Determine if there are any empty rows.
pref_cities.count()

City ID                127
City                   127
Lat                    127
Lng                    127
Max Temp               127
Humidity               127
Cloudiness             127
Wind Speed             127
Country                125
Current Description    127
Date                   127
dtype: int64

In [46]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
pref_cities=pref_cities.dropna()
pref_cities.count()


City ID                125
City                   125
Lat                    125
Lng                    125
Max Temp               125
Humidity               125
Cloudiness             125
Wind Speed             125
Country                125
Current Description    125
Date                   125
dtype: int64

In [47]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = pref_cities[["City", "Country", "Max Temp","Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
5,Thompson,CA,60.96,overcast clouds,55.7435,-97.8558,
6,Nikolskoye,RU,60.69,broken clouds,59.7035,30.7861,
15,San Quintin,MX,65.57,few clouds,30.4833,-115.9500,
23,Lagoa,PT,63.82,overcast clouds,39.0500,-27.9833,
29,Mackay,AU,66.04,overcast clouds,-21.1500,149.2000,
31,Polis,CY,66.02,clear sky,35.0367,32.4264,
33,Cheran,MX,64.80,overcast clouds,19.6833,-101.9500,
39,Byron Bay,AU,63.05,few clouds,-28.6500,153.6167,
40,Castro,BR,60.35,overcast clouds,-24.7911,-50.0119,
45,Berlin,DE,68.07,clear sky,52.5244,13.4105,


In [48]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    params["location"] = f"{lat},{lng}"
    
    hotels = requests.get(base_url, params=params).json()

    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [ ]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df=hotel_df[hotel_df['Hotel Name']!=""]
clean_hotel_df

In [25]:
# 8a. Create the output File (CSV)
output_data_file = "../Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [51]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
<dt>Current Weather</dt><dd>{Current Description}</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [52]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))